# Webscraping & Nettoyage
# Ce notebook regroupe uniquement :
# 1) la collecte (web scraping) des articles Gaza/Ukraine
# 2) le nettoyage minimal et la sauvegarde en JSON


Web scrapping gaza


In [7]:
# web scrapping gaza 
import requests
from bs4 import BeautifulSoup
import json
import time
from datetime import datetime
from requests.adapters import HTTPAdapter
from urllib3.util import Retry

# --- LISTE DES URLS NETTOYÉE ET COMPLÈTE ---
urls_gaza = [
    # --- Fox News ---
    "https://www.foxnews.com/world/israeli-military-sees-hamas-war-lasting-all-2024-report",
    "https://www.foxnews.com/world/idf-seized-weapons-cache-supplies-hamas-terror-groups-attack-israel",
    "https://www.foxnews.com/world/israeli-defense-forces-release-video-showing-evidence-hamas-weapons-tunnels-linking-hospital-basements",
    "https://www.foxnews.com/world/israeli-troops-enter-al-shifa-hospital-killing-hamas-terrorists-gate-side-firefight",
    "https://www.foxnews.com/world/urban-warfare-expert-says-israeli-military-taking-unprecedented-steps-to-protect-gaza-civilians",
    "https://www.foxnews.com/world/idf-investigating-whether-hamas-chief-yahya-sinwar-killed-gaza-operations",
    "https://www.foxnews.com/world/hamas-command-tunnel-unrwas-headquarters-gaza-israel-says",
    "https://www.foxnews.com/world/anti-hamas-protests-breakout-gaza-strip-over-demands-end-war",
    "https://www.foxnews.com/politics/trump-warns-hamas-war-resume-terror-group-continues-kill-people-gaza",
    "https://www.foxnews.com/world/israel-says-hamas-violated-ceasefire-multiple-attacks-leading-idf-response",
    "https://www.foxnews.com/world/study-disputes-gaza-genocide-charges-finds-flawed-data-amid-hamas-driven-narrative",
    "https://www.foxnews.com/world/ceasefire-over-israel-strikes-gaza-after-hamas-refused-release-hostages-officials-say",
    "https://www.foxnews.com/world/taking-out-hamas-million-dollar-root-tunnel-game-changer-analyst-says",
    "https://www.foxnews.com/world/hamas-hardens-stance-ceasefire-talks-netanyahu-recalls-negotiation-team",
    "https://www.foxnews.com/world/former-hamas-hostage-warned-australian-leaders-about-dangers-antisemitism-months-before-bondi-beach-attack",
    "https://www.foxnews.com/world/hamas-faces-legitimacy-crisis-desperate-gazans-flock-us-backed-aid-centers",
    "https://www.foxnews.com/world/israel-constructs-large-pumps-flood-hamas-terror-tunnels-seawater-wall-street-journal",
    "https://www.foxnews.com/world/gaza-hospital-blast-heres-what-we-know",
    "https://www.foxnews.com/world/netanyahu-announces-investigation-tragic-mishap-reports-journalists-killed-gaza-strike",
    # --- NOUVELLES URLs FOX NEWS (NETTOYÉES) ---
    "https://www.foxnews.com/world/israel-security-cabinet-approves-plan-occupy-gaza-city",
    "https://www.foxnews.com/world/i-will-be-haunted-forever-israels-horrific-video-hamas-atrocities-leaves-viewers-shocked-sickened",
    "https://www.foxnews.com/world/israel-under-attack-gunmen-kill-injure-tel-aviv",
    "https://www.foxnews.com/world/reporters-notebook-israeli-hostage-families-plead-peace-falters-war-grinds-gaza",
    "https://www.foxnews.com/world/report-exposes-hamas-terrorist-crimes-against-families-during-oct-7-massacre-kinocide",
    "https://www.foxnews.com/world/un-finally-recognizes-israeli-women-raped-sexually-attacked-hamas-terrorists",
    "https://www.foxnews.com/politics/hochul-apologizes-saying-israel-right-defend-itself-inappropriate-canada-analogy",
    "https://www.foxnews.com/world/israel-war-see-photos-video-attack-aftermath",
    "https://www.foxnews.com/world/powerful-israel-backed-clan-leader-fought-hamas-murdered-gaza",
    "https://www.foxnews.com/world/grassroots-push-freedom-grows-gaza-hamas-tightens-its-deadly-grip",
    "https://www.foxnews.com/world/multiple-people-killed-shooting-attack-jerusalem-bus",
    # --- BBC ---
    "https://www.bbc.com/news/articles/clyv7w3gdy2o",
    "https://www.bbc.com/news/world-middle-east-67480680",
    "https://www.bbc.com/news/articles/c4gdx1rpel6o",
    "https://www.bbc.com/news/articles/cx2dklj3rxvo",
    "https://www.bbc.com/news/articles/c17x0xjqj21o",
    "https://www.bbc.com/news/articles/c0rvxjnvv71o",
    "https://www.bbc.com/news/articles/cvgn50d9pywo",
    "https://www.bbc.com/news/articles/cr54y0qplgvo",
    "https://www.bbc.com/news/world-middle-east-67401064",
    "https://www.bbc.com/news/world-middle-east-67316463",
    "https://www.bbc.com/news/articles/cjr7l123zy5o",
    "https://www.bbc.com/news/articles/crlzr7ey8rpo",
    "https://www.bbc.com/news/articles/cr5e4ee9r13o",
    "https://www.bbc.com/news/articles/cnvezzdnmrno",
    "https://www.bbc.com/news/articles/czry8g5n80zo",
    "https://www.bbc.com/news/articles/c86wd84y7jyo",
    "https://www.bbc.com/news/articles/c0lk292jww4o",
    "https://www.bbc.com/news/articles/c15wz2ee05do",
    # --- The Guardian ---
    "https://www.theguardian.com/world/2025/jan/15/the-devastating-impact-of-15-months-of-war-on-gaza",
    "https://www.theguardian.com/world/2025/oct/07/diplomacy-lowest-point-how-the-israel-gaza-conflict-was-mishandled",
    # --- CNN ---
    "https://edition.cnn.com/2023/12/21/middleeast/palestinians-back-hamas-survey-intl-cmd",
    "https://edition.cnn.com/2025/07/30/middleeast/gaza-women-dangers-starvation-children-intl-cmd",
    "https://edition.cnn.com/2025/05/20/middleeast/gaza-hunger-jana-israel-blockade-intl-cmd",
    "https://edition.cnn.com/2025/07/26/middleeast/five-month-old-dies-gaza-starvation-intl",
    "https://edition.cnn.com/2025/03/18/world/israels-strikes-in-gaza-palestinian-children-traumatized",
    # --- AP News ---
    "https://apnews.com/article/israel-hamas-gaza-palestinians-war-news-19-12-2025-c5cbf81d48dd1ee82b1fcd2706e7bc7a"
]

corpus_gaza = []

# --- 1. MISE EN PLACE D'UNE STRATÉGIE DE RETRY ---
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
print("=" * 60)
print("DÉBUT DE LA COLLECTE DU CORPUS GAZA")
print(f"Nombre total d'URLs à traiter : {len(urls_gaza)}")
print("=" * 60)

session = requests.Session()
retry_strategy = Retry(
    total=3,
    backoff_factor=2,
    status_forcelist=[429, 500, 502, 503, 504]
)
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("https://", adapter)

# --- 2. BOUCLE PRINCIPALE ---
article_count = 0
error_count = 0
success_count = 0

for index, url in enumerate(urls_gaza, 1):
    try:
        print(f"\n[{index}/{len(urls_gaza)}] Traitement de : {url[:60]}...")
        r = session.get(url, headers=headers, timeout=(5, 30))
        if r.status_code == 200:
            soup = BeautifulSoup(r.text, 'html.parser')
            # --- 1. Titre ---
            title_tag = soup.find('h1')
            titre = title_tag.get_text(strip=True) if title_tag else "Sans titre"
            # --- 2. Détection Source & Nettoyage spécifique ---
            source = "Autre"
            author = "N/A"
            publish_date = "N/A"
            article_body = None
            if "foxnews.com" in url:
                source = "Fox News"
                article_body = soup.find('div', class_='article-body')
                author_tag = soup.find('div', class_='author-byline')
                if author_tag:
                    author = author_tag.get_text(strip=True).replace('By ', '')
                date_tag = soup.find('time')
                if date_tag:
                    publish_date = date_tag.get_text(strip=True)
            elif "bbc.com" in url:
                source = "BBC"
                article_body = soup.find('article')
                date_tag = soup.find('time')
                if date_tag:
                    publish_date = date_tag.get('datetime')
            elif "theguardian.com" in url:
                source = "The Guardian"
                article_body = soup.find('div', class_='article-body-commercial-selector')
                author_tag = soup.find('a', rel='author')
                if author_tag:
                    author = author_tag.get_text(strip=True)
                date_tag = soup.find('time')
                if date_tag:
                    publish_date = date_tag.get('datetime')
            elif "cnn.com" in url:
                source = "CNN"
                article_body = soup.find('section', attrs={'data-zone-label': 'bodyText'}) or soup.find('div', class_='article__content')
                if not article_body:
                    article_body = soup.find('div', {'class': 'l-container'})
                date_tag = soup.find('div', class_='timestamp')
                if date_tag:
                    publish_date = date_tag.get_text(strip=True)
            elif "apnews.com" in url or "reuters.com" in url:
                source = "AP/Reuters" if "apnews.com" in url else "Reuters"
                article_body = soup.find('article') or soup.find('div', {'role': 'article'})
                date_tag = soup.find('time')
                if date_tag:
                    publish_date = date_tag.get('datetime') or date_tag.get_text(strip=True)
            else:
                source = "Autre"
                article_body = soup.find('article')
            # --- 3. Extraction du texte ---
            paragraphes = []
            if article_body:
                paragraphes = [p.get_text(strip=True) for p in article_body.find_all('p') if len(p.get_text(strip=True)) > 50]
                if len(paragraphes) < 3:
                    text_elements = article_body.find_all(['p', 'div'], class_=lambda x: x and ('text' in x or 'content' in x or 'paragraph' in x))
                    paragraphes = [elem.get_text(strip=True) for elem in text_elements if len(elem.get_text(strip=True)) > 50]
            else:
                main_content = soup.find('main') or soup.find('div', {'role': 'main'})
                if main_content:
                    paragraphes = [p.get_text(strip=True) for p in main_content.find_all('p') if len(p.get_text(strip=True)) > 50]
                else:
                    all_paragraphs = soup.find_all('p')
                    for p in all_paragraphs:
                        text = p.get_text(strip=True)
                        if len(text) > 80 and "©" not in text and "cookie" not in text.lower() and "privacy" not in text.lower():
                            paragraphes.append(text)
            texte_final = "\n".join(paragraphes)
            # --- 4. Ajout au corpus ---
            if len(texte_final) > 100:
                corpus_gaza.append({
                    "source": source,
                    "url": url,
                    "title": titre,
                    "publish_date": publish_date,
                    "author": author,
                    "content": texte_final,
                    "scraped_at": datetime.now().isoformat(),
                    "keywords": [],
                    "summary": "",
                    "conflict": "Gaza"
                })
                article_count += 1
                success_count += 1
                print(f"   ✅ [{article_count}] Succès : {titre[:60]}... ({source})")
            else:
                error_count += 1
                print(f"   ❌ Contenu trop court ou vide pour : {url[:40]}...")
        else:
            error_count += 1
            print(f"   ❌ Erreur {r.status_code} pour l'URL")
        time.sleep(3)
    except Exception as e:
        error_count += 1
        print(f"   ⚠️  Erreur critique : {str(e)[:80]}")

# --- 3. SAUVEGARDE --- dans le docier corpus 
file_name = 'corpus/corpus_gaza_v1.json'
with open(file_name, 'w', encoding='utf-8') as f:
    json.dump(corpus_gaza, f, indent=4, ensure_ascii=False)

print("\n" + "=" * 60)
print("COLLECTE TERMINÉE")
print("=" * 60)
print(f"📊 RÉSULTATS :")
print(f"   • Articles collectés avec succès : {success_count}/{len(urls_gaza)}")
print(f"   • Échecs/erreurs : {error_count}")
print(f"   • Total dans le corpus : {article_count} articles")
print(f"   • Taille du fichier JSON : {file_name}")
print("=" * 60)

try:
    from google.colab import files
    files.download(file_name)
    print("📥 Fichier téléchargé depuis Google Colab")
except ImportError:
    print(f"📁 Fichier disponible localement : {file_name}")

DÉBUT DE LA COLLECTE DU CORPUS GAZA
Nombre total d'URLs à traiter : 56

[1/56] Traitement de : https://www.foxnews.com/world/israeli-military-sees-hamas-wa...
   ✅ [1] Succès : Israeli military sees Hamas war lasting through all of 2024:... (Fox News)

[2/56] Traitement de : https://www.foxnews.com/world/idf-seized-weapons-cache-suppl...
   ✅ [2] Succès : IDF seized weapons cache, supplies from Hamas during terror ... (Fox News)

[3/56] Traitement de : https://www.foxnews.com/world/israeli-defense-forces-release...
   ✅ [3] Succès : Israeli Defense Forces release video showing evidence of Ham... (Fox News)

[4/56] Traitement de : https://www.foxnews.com/world/israeli-troops-enter-al-shifa-...
   ✅ [4] Succès : Israeli troops enter Al-Shifa Hospital after killing Hamas t... (Fox News)

[5/56] Traitement de : https://www.foxnews.com/world/urban-warfare-expert-says-isra...
   ✅ [5] Succès : Urban warfare expert says Israeli military taking unpreceden... (Fox News)

[6/56] Traitement de : 

Web scrapping Ukrain 

In [9]:
# web scraping Ukraine 

import requests
from bs4 import BeautifulSoup
import json
import time
from datetime import datetime
from requests.adapters import HTTPAdapter
from urllib3.util import Retry

# --- LISTE DES URLS UKRAINE ---
urls_ukraine = [
    "https://www.foxnews.com/politics/melania-trump-says-7-more-ukrainian-children-reunited-families-part-initiative-russia",
    "https://www.foxnews.com/world/ukraines-underground-railroad-rescues-abducted-ukrainian-children-from-russian-reeducation-camps",
    "https://www.foxnews.com/lifestyle/humanitarian-aid-expert-rescue-children-ukraine",
    "https://www.foxnews.com/lifestyle/unicef-gives-aid-russia-ukraine-refugees",
    "https://www.foxnews.com/lifestyle/ukraine-refugee-children-war-help-kids",
    "https://www.foxnews.com/world/thousands-ukrainian-children-forcibly-deported-russia-july-us-security-official-says",
    "https://www.foxnews.com/world/ukrainian-children-death-toll-humanitarian-crisis-food",
    "https://www.foxnews.com/politics/just-evil-top-republican-details-russias-horrific-mass-abductions-ukrainian-children",
    "https://www.foxnews.com/politics/trump-administration-ends-program-track-kidnapped-ukrainian-children-russia-lawmakers-say",
    "https://www.foxnews.com/opinion/sens-klobuchar-grassley-america-cant-ignore-russia-kidnapping-ukrainian-children",
    "https://www.foxnews.com/world/rescuers-continue-search-kyiv-childrens-hospital-hit-russian-missile",
    "https://www.foxnews.com/lifestyle/children-traumatized-war-ukraine-mentors-places",
    "https://www.foxnews.com/lifestyle/ukrainian-children-cancer-evacuated-poland-war",
    "https://www.foxnews.com/world/ukraines-top-prosecutor-speaks-evil-russian-atrocities",
    "https://www.foxnews.com/world/half-ukraine-children-displaced",
    "https://www.foxnews.com/us/tennessee-first-state-receive-child-cancer-patients-ukraine",
    "https://www.foxnews.com/lifestyle/premature-babies-ukraine-rescued-ambulance-kyiv-shelling",
    "https://www.foxnews.com/entertainment/jessica-chastain-shares-footage-life-changing-ukraine-trip-displaced-children-destruction",
    "https://www.foxnews.com/media/ukrainian-mother-fleeing-country-with-her-son-details-escape-i-wish-i-never-experienced-that",
    "https://www.foxnews.com/lifestyle/franklin-graham-ukraine-war-children",
    "https://www.foxnews.com/world/ukraines-foreign-minister-accuses-russia-war-crimes-attacks-school-orphanage",
    "https://www.foxnews.com/world/world-leaders-react-to-bucha-ukraine-massacre-russia-blamed",
    "https://www.foxnews.com/world/strikes-ukraine-hospital-kills-2-day-old-baby-officials-say",
    "https://www.foxnews.com/world/300-dead-mariupol-theater-russian-attacks",
    "https://www.foxnews.com/world/ukraine-children-killed-wounded-russian-invasion",
    "https://www.foxnews.com/world/refugees-fleeing-ukraine-reaches-2-5-million-many-children-un-says",
    "https://www.foxnews.com/world/nearly-50-children-from-russian-occupied-regions-ukraine-arrive-belarus",
    "https://www.foxnews.com/world/ukraine-rescue-team-brings-back-31-children-russia-war",
    "https://www.foxnews.com/world/russian-attacks-on-ukrainian-hospitals-could-be-war-crimes-lawyer-says",
    "https://www.foxnews.com/world/russia-denies-bombed-childrens-hospital-fake-news",
    "https://www.foxnews.com/world/ukrainian-lawmaker-russia-trying-forcibly-deport-civilians-mariupol",
    "https://www.bbc.com/news/articles/c62vk0v9756o",
    "https://edition.cnn.com/2025/12/24/europe/moscow-bomb-police-officers-intl",
    "https://edition.cnn.com/2025/12/23/europe/russian-attack-zelensky-ukraine-talks-intl",
    "https://edition.cnn.com/2025/12/10/politics/ukraine-response-peace-plan",
    "https://www.bbc.com/news/world-europe-60633482",
    "https://amp.cnn.com/cnn/2024/06/13/europe/russia-ukraine-mariupol-hunger-war-crime-intl",
    "https://amp.cnn.com/cnn/2022/03/08/europe/russia-invasion-ukraine-03-08-intl",
    "https://www.theguardian.com/uk-news/2025/jun/27/ukrainians-who-fled-to-uk-being-refused-asylum-on-grounds-it-is-safe-to-return",
    # --- Nouveaux liens validés (Militaire et Stratégique) ---
    "https://www.bbc.com/news/world-europe-68806207",
    "https://www.bbc.com/news/world-europe-60506682",
    "https://apnews.com/article/russia-ukraine-war-kharkiv-frontline-drones-f66f9a0c44369a8468673f8e5f22e413",
    "https://www.foxnews.com/world/ukraine-counteroffensive-military-strategy-russia-war",
    "https://www.reuters.com/world/europe/putin-says-russia-fired-high-hypersonic-ballistic-missile-at-ukraine-2024-11-21/",
    "https://www.reuters.com/world/europe/us-halts-some-missile-shipments-ukraine-over-low-stockpiles-2024-07-01/",
    "https://www.reuters.com/business/aerospace-defense/us-nato-developing-novel-funding-mechanism-ukraine-weapons-transfers-2024-08-01/",
    "https://www.theguardian.com/world/2025/mar/28/ukraine-war-briefing-another-15m-artillery-shells-for-ukraine",
    "https://www.foxnews.com/politics/zelenskyy-names-his-terms-giving-up-power-us-ukraine-lock-mineral-deal-nato",
    "https://apnews.com/article/russia-ukraine-war-interceptors-drones-nato-c7b57962e573b344490b07b2cfead856",
    "https://www.reuters.com/world/europe/ukraine-says-russian-missile-hit-childrens-hospital-kyiv-2024-07-08/",
    "https://www.bbc.com/news/world-europe-61266843",
    "https://edition.cnn.com/2023/08/28/economy/russia-military-spending-economic-impact",
    "https://www.reuters.com/world/europe/russia-says-it-captured-town-east-ukraine-2024-07-31/",
    "https://apnews.com/article/russia-ukraine-war-kremlin-dmitry-peskov-a36f04fbf04b7ff702920f75fade3789"
]

corpus_ukraine = []

# --- 1. CONFIGURATION SESSION ---
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
print("=" * 60)
print("DÉBUT DE LA COLLECTE DU CORPUS UKRAINE")
print(f"Nombre total d'URLs à traiter : {len(urls_ukraine)}")
print("=" * 60)

session = requests.Session()
retry_strategy = Retry(
    total=3,
    backoff_factor=2,
    status_forcelist=[429, 500, 502, 503, 504]
)
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("https://", adapter)

# --- 2. BOUCLE PRINCIPALE ---
article_count = 0
error_count = 0

for index, url in enumerate(urls_ukraine, 1):
    try:
        print(f"\n[{index}/{len(urls_ukraine)}] Analyse de : {url[:60]}...")
        
        r = session.get(url, headers=headers, timeout=(5, 30))
        
        if r.status_code == 200:
            soup = BeautifulSoup(r.text, 'html.parser')
            
            # --- Extraction Titre ---
            title_tag = soup.find('h1')
            titre = title_tag.get_text(strip=True) if title_tag else "Sans titre"
            
            # --- Logique par Source ---
            source = "Autre"
            author = "N/A"
            publish_date = "N/A"
            article_body = None
            
            if "foxnews.com" in url:
                source = "Fox News"
                article_body = soup.find('div', class_='article-body')
                author_tag = soup.find('div', class_='author-byline')
                if author_tag: author = author_tag.get_text(strip=True).replace('By ', '')
                date_tag = soup.find('time')
                if date_tag: publish_date = date_tag.get_text(strip=True)
            
            elif "bbc.com" in url:
                source = "BBC"
                article_body = soup.find('article')
                date_tag = soup.find('time')
                if date_tag: publish_date = date_tag.get('datetime')
            
            elif "theguardian.com" in url:
                source = "The Guardian"
                article_body = soup.find('div', class_='article-body-commercial-selector') or soup.find('div', id='maincontent')
                date_tag = soup.find('time')
                if date_tag: publish_date = date_tag.get('datetime')
            
            elif "cnn.com" in url:
                source = "CNN"
                article_body = soup.find('section', attrs={'data-zone-label': 'bodyText'}) or soup.find('div', class_='article__content')
                date_tag = soup.find('div', class_='timestamp')
                if date_tag: publish_date = date_tag.get_text(strip=True)

            # --- Extraction du texte ---
            paragraphes = []
            target = article_body if article_body else soup
            
            paragraphes = [p.get_text(strip=True) for p in target.find_all('p') 
                          if len(p.get_text(strip=True)) > 40 and "©" not in p.get_text()]
            
            texte_final = "\n".join(paragraphes)
            
            if len(texte_final) > 150:
                corpus_ukraine.append({
                    "source": source,
                    "url": url,
                    "title": titre,
                    "publish_date": publish_date,
                    "author": author,
                    "content": texte_final,
                    "scraped_at": datetime.now().isoformat(),
                    "conflict": "Ukraine"
                })
                article_count += 1
                print(f"   ✅ [{article_count}] Succès : {titre[:50]}...")
            else:
                print(f"   ❌ REJETÉ : Contenu trop court ({len(texte_final)} car.) pour l'URL : {url}")
                error_count += 1
        else:
            print(f"   ❌ Erreur HTTP {r.status_code}")
            error_count += 1
        
        # Respect des serveurs
        time.sleep(2)
        
    except Exception as e:
        print(f"   ⚠️ Erreur : {str(e)[:50]}")
        error_count += 1

# --- 3. SAUVEGARDE  dans le docier corpus ---
file_name = 'corpus/corpus_ukraine_v1.json'
with open(file_name, 'w', encoding='utf-8') as f:
    json.dump(corpus_ukraine, f, indent=4, ensure_ascii=False)

print("\n" + "=" * 60)
print("COLLECTE UKRAINE TERMINÉE")
print(f"📊 TOTAL : {article_count} articles sauvegardés dans {file_name}")
print(f"⚠️ ÉCHECS : {error_count}")
print("=" * 60)

DÉBUT DE LA COLLECTE DU CORPUS UKRAINE
Nombre total d'URLs à traiter : 54

[1/54] Analyse de : https://www.foxnews.com/politics/melania-trump-says-7-more-u...
   ✅ [1] Succès : Melania Trump says 7 more Ukrainian children reuni...

[2/54] Analyse de : https://www.foxnews.com/world/ukraines-underground-railroad-...
   ✅ [2] Succès : Ukraine’s 'Underground Railroad' rescues abducted ...

[3/54] Analyse de : https://www.foxnews.com/lifestyle/humanitarian-aid-expert-re...
   ✅ [3] Succès : Humanitarian aid expert reveals dramatic rescue of...

[4/54] Analyse de : https://www.foxnews.com/lifestyle/unicef-gives-aid-russia-uk...
   ✅ [4] Succès : Half a million children become refugees as Russia-...

[5/54] Analyse de : https://www.foxnews.com/lifestyle/ukraine-refugee-children-w...
   ✅ [5] Succès : Ukraine's refugee children: Amid the war, how to h...

[6/54] Analyse de : https://www.foxnews.com/world/thousands-ukrainian-children-f...
   ✅ [6] Succès : 'Thousands' of Ukrainian children forc